In [1]:
from __future__ import unicode_literals, division
import re
import sys
from tqdm import tqdm_notebook

In [2]:
# ! pip install tqdm --user 

In [3]:
import pandas as pd
import numpy as np
import spacy

import tensorflow as tf

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (10,10)
sns.set()

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from embedding import get_embedding
from config import Config
from data_utils import tokenize_sent

/home/u26008/.local/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


## POS Tags

In [7]:
nlp = spacy.load('en_core_web_sm')
def get_pos_tags(word_list):
    tags = []
    sent = ' '.join(word_list)
    doc = nlp(sent)
    [tags.append(token.tag_) for token in doc]
    return tags

In [8]:
get_pos_tags('i druuink tea'.split())

['PRP', 'VBP', 'NN']

# Get configurations

In [9]:
config = Config()
max_len = 70
embedding_size = config.embedding_size

In [10]:
path = '../train.csv'

In [22]:
data = pd.read_csv(path)


In [23]:
data.drop_duplicates(subset=['question1','question2'], inplace=True)

In [24]:
data.question1 = data.question1.apply(lambda x: tokenize_sent(str(x).lower()))
data.question2 = data.question2.apply(lambda x: tokenize_sent(str(x).lower()))

In [38]:
sentences = np.concatenate((data.question1.values, data.question2.values))

In [67]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.test.utils import common_texts, get_tmpfile

In [40]:
model = Word2Vec(sentences=sentences)

In [45]:
from nltk.cluster import KMeansClusterer

In [53]:
X = model.wv[model.wv.vocab]

In [ ]:
NUM_CLUSTERS=10
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

In [57]:
# print (assigned_clusters)

In [62]:
d = pd.DataFrame(data={'words':list(model.wv.vocab), 'cluster': assigned_clusters})

In [66]:
# i+=1
# d[d['cluster']==i]

In [ ]:
KeyedVectors.save_word2vec_format(model.wv)

In [68]:
path = get_tmpfile('w2v.wv')

In [74]:
model.wv.save_word2vec_format('../w2v.wv', binary=True)

In [70]:
m = KeyedVectors.load_word2vec_format('w2v.wv')

In [72]:
# m['what']